## Some charts about COVID-19 
Provided daily via Telegram channel: https://t.me/covid19charts. You can subscribe and receive automatic updates.

Some basics charts about COVID-19 from https://github.com/CSSEGISandData/COVID-19.git
**Cumulative, daily and percentage** charts on **Region/Country** with more than **500 Confirmed**. 

Code available at https://github.com/andreaaizza/covid-19-basic-charts

In [ ]:
import pandas as pd
df_c = pd.read_csv(
    "./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_c=df_c.drop(columns=['Lat','Long'])
df_c=df_c.groupby('Country/Region').sum()

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# load Confirmed to filter
df_c = pd.read_csv(
    "./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv")
df_c=df_c.drop(columns=['Lat','Long'])
df_c=df_c.groupby('Country/Region').sum()

# get only >500
last_col=df_c.columns[len(df_c.columns)-1]
significant_ones=df_c[df_c[last_col]>500].index

# load all files
df_suff=['Confirmed','Deaths','Recovered']
df_suff_without_confirmed=['Deaths','Recovered']
df=pd.DataFrame()
for s in df_suff:
    d=pd.read_csv(
    "./COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-"+s+".csv")
    d=d.drop(columns=['Lat','Long'])
    d=d.groupby('Country/Region').sum()
    d['Type']=s
    df=df.append(d)
# set multiindex
df=df.set_index('Type', append=True)

### Cumulative Confirmed only

In [ ]:
fig = go.Figure()
for l in significant_ones:
    if l!='Mainland China' and l!='Others':
        fig.add_trace(
            go.Scatter(
                x=df.columns, 
                y=df_c[df_c.index==l].iloc[0],
                name=l,
                mode='lines+markers'
            )
        )
fig.update_layout(xaxis_rangeslider_visible=True)
fig.show()

In [ ]:
def plotCounties(title, f): 
    plt_size=len(significant_ones)
    for i in range(1,plt_size): 
        country=significant_ones[i]
        fig = go.Figure()
        for t in df_suff:
            f(df, fig, i, t, country)
        fig.update_layout(title_text=country)
        fig.show()

### Cumulative

In [ ]:
def p_cum(df, fig, i, t, country):
    fig.add_trace(
        go.Scatter(
            x=df.columns, 
            y=df[
                (df.index.get_level_values('Type')==t) & 
                (df.index.get_level_values('Country/Region')==country)
            ].iloc[0],
            name=t,
            mode='lines+markers'
        )
    )
    
plotCounties('Cumulative', p_cum)

### Daily

In [ ]:
def p_daily(df, fig, i, t, country):
    data=df[
        (df.index.get_level_values('Type')==t) & 
        (df.index.get_level_values('Country/Region')==country)].iloc[0]
    fig.add_trace(
        go.Scatter(
            x=df.columns, 
            y=data-data.shift(1),
            name=t,
            mode='lines+markers'
        )
    )
    
plotCounties('Daily', p_daily)

### Daily as percentage of Confirmed

In [ ]:
def p_ppcc(df, fig, i, t, country):
    data_confirmed=df[
        (df.index.get_level_values('Type')=='Confirmed') & 
        (df.index.get_level_values('Country/Region')==country)].iloc[0]
    data=df[
        (df.index.get_level_values('Type')==t) & 
        (df.index.get_level_values('Country/Region')==country)].iloc[0]
    data_delta=data-data.shift(1)
    fig.add_trace(
        go.Scatter(
            x=df.columns, 
            y=data_delta/data_confirmed,
            name=t,
            mode='lines+markers'
        )
    )
    
plotCounties('Daily as percentage of Confirmed', p_ppcc)